In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e' or base == 'Delta':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [3]:
simp = True

## Target function

we want to minimize the effective price, given a set of Deltas for the buying and selling asset each.

In [4]:
def spotByDelta_(asset, s, v, b, w, j, e, Delta, a, min):
  if asset == 'b':
    d = -Delta
  else:
    d = Delta
  f = (v + b + d) * w
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
spotByDelta['b']

w_b*(-Delta_b + b_b + v_b)

In [5]:
spotByDelta['s']

w_s*(Delta_s + b_s + v_s)

In [6]:
def expBySpot_(s, v, b, w, j, e, Delta, a, min):
  f = log(s/a) / log(1 + 1/j)
  if simp:
    f = simplify(f)
  return f
expBySpot = {asset: expBySpot_(*all_symbols[asset]) for asset in assets}
expBySpot['b']

log((s_b/a_b)**(1/log((j_b + 1)/j_b)))

In [7]:
expBySpot['s']

log((s_s/a_s)**(1/log((j_s + 1)/j_s)))

In [8]:
def expByDelta_(asset, s, v, b, w, j, e, Delta, a, min):
    f = expBySpot[asset].subs(s, spotByDelta[asset])
    if simp:
        f = simplify(f)
    return f

expByDelta = {asset: expByDelta_(asset, *all_symbols[asset]) for asset in assets}
expByDelta['b']

log((w_b*(-Delta_b + b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))

In [9]:
eff = Delta_s / Delta_b
eff

Delta_s/Delta_b

In [10]:
Delta_b_by_Delta_s = Delta_s * spotByDelta['b'] / spotByDelta['s']
if simp:
    Delta_b_by_Delta_s = simplify(Delta_b_by_Delta_s)
Delta_b_by_Delta_s

Delta_s*w_b*(-Delta_b + b_b + v_b)/(w_s*(Delta_s + b_s + v_s))

In [11]:
Delta_s_by_Delta_b = Delta_b * spotByDelta['s'] / spotByDelta['b']
if simp:
  Delta_s_by_Delta_b = simplify(Delta_s_by_Delta_b)
Delta_s_by_Delta_b

Delta_b*w_s*(Delta_s + b_s + v_s)/(w_b*(-Delta_b + b_b + v_b))

In [12]:
effByDelta_b = eff.subs(Delta_s, Delta_s_by_Delta_b)
if simp:
  effByDelta_b = simplify(effByDelta_b)
effByDelta_b

w_s*(Delta_s + b_s + v_s)/(w_b*(-Delta_b + b_b + v_b))

In [13]:
effByDelta_s = eff.subs(Delta_b, Delta_b_by_Delta_s)
if simp:
  effByDelta_s = simplify(effByDelta_s)
effByDelta_s

w_s*(Delta_s + b_s + v_s)/(w_b*(-Delta_b + b_b + v_b))

In [14]:
simplify(effByDelta_s - effByDelta_b)

0

In [15]:
effByDeltas = effByDelta_b

In [16]:
H = hessian(effByDeltas, [Delta_b, Delta_s])
H

Matrix([
[2*w_s*(Delta_s + b_s + v_s)/(w_b*(-Delta_b + b_b + v_b)**3), w_s/(w_b*(-Delta_b + b_b + v_b)**2)],
[                        w_s/(w_b*(-Delta_b + b_b + v_b)**2),                                   0]])

In [17]:
topLeft = simplify(H[0, 0])
topLeft

2*w_s*(Delta_s + b_s + v_s)/(w_b*(-Delta_b + b_b + v_b)**3)

-> positive, because Delta_b <= b_b

In [18]:
det = H.det()
det = simplify(det)
det

-w_s**2/(w_b**2*(Delta_b**4 - 4*Delta_b**3*b_b - 4*Delta_b**3*v_b + 6*Delta_b**2*b_b**2 + 12*Delta_b**2*b_b*v_b + 6*Delta_b**2*v_b**2 - 4*Delta_b*b_b**3 - 12*Delta_b*b_b**2*v_b - 12*Delta_b*b_b*v_b**2 - 4*Delta_b*v_b**3 + b_b**4 + 4*b_b**3*v_b + 6*b_b**2*v_b**2 + 4*b_b*v_b**3 + v_b**4))

In [19]:
len(det.args)

4

In [20]:
det.args[0]

-1

In [21]:
det.args[1]

w_b**(-2)

In [22]:
det.args[2]

w_s**2

In [23]:
det.args[3]

1/(Delta_b**4 - 4*Delta_b**3*b_b - 4*Delta_b**3*v_b + 6*Delta_b**2*b_b**2 + 12*Delta_b**2*b_b*v_b + 6*Delta_b**2*v_b**2 - 4*Delta_b*b_b**3 - 12*Delta_b*b_b**2*v_b - 12*Delta_b*b_b*v_b**2 - 4*Delta_b*v_b**3 + b_b**4 + 4*b_b**3*v_b + 6*b_b**2*v_b**2 + 4*b_b*v_b**3 + v_b**4)

In [24]:
det.args[3].func

sympy.core.power.Pow

In [25]:
len(det.args[3].args)

2

In [26]:
det.args[3].args[1]

-1

In [27]:
f = det.args[3].args[0]
f

Delta_b**4 - 4*Delta_b**3*b_b - 4*Delta_b**3*v_b + 6*Delta_b**2*b_b**2 + 12*Delta_b**2*b_b*v_b + 6*Delta_b**2*v_b**2 - 4*Delta_b*b_b**3 - 12*Delta_b*b_b**2*v_b - 12*Delta_b*b_b*v_b**2 - 4*Delta_b*v_b**3 + b_b**4 + 4*b_b**3*v_b + 6*b_b**2*v_b**2 + 4*b_b*v_b**3 + v_b**4

In [28]:
solve(Eq(f, 0)) # -> can never happen because Delta_b <= b_b

[{Delta_b: b_b + v_b}]

In [29]:
f.subs(Delta_b, b_b)

v_b**4

In [30]:
f.subs(Delta_b, 0)

b_b**4 + 4*b_b**3*v_b + 6*b_b**2*v_b**2 + 4*b_b*v_b**3 + v_b**4

In [31]:
df = f.diff(Delta_b)
df

4*Delta_b**3 - 12*Delta_b**2*b_b - 12*Delta_b**2*v_b + 12*Delta_b*b_b**2 + 24*Delta_b*b_b*v_b + 12*Delta_b*v_b**2 - 4*b_b**3 - 12*b_b**2*v_b - 12*b_b*v_b**2 - 4*v_b**3

In [32]:
solve(Eq(df, 0)) # -> can never happen because Delta_b <= b_b

[{Delta_b: b_b + v_b}]

-> since the boundaries are both positive, and the derivative is only zero at one point outside the interval, the function is positive everywhere

-> determinant is negative

-> neither convex nor concave / has a saddle-point

-> might get multiple solutions which might even not include the local optimum

## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

In [ ]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    exp = expByDelta[asset]
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = exp - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - exp # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

In [ ]:
expBound['s']

In [ ]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a, min):
  return spotByDelta[asset] - I_max # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

In [ ]:
def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    f = min - Delta # <=! 0
    if simp:
        f = simplify(f)
    return f

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

In [ ]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        max = b
    else:
        max = max_s
    f = Delta - max # <=! 0
    if simp:
        f = simplify(f)
    return f

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

In [ ]:
maxAmntBound['s']

## Lagrangian

In [ ]:
l1, l2, l3, l4, l5, l6, l7, l8 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6 lambda_7 lambda_8', positive=True)

L = eff + l1 * expBound['b'] + l2 * expBound['s'] + l3 * maxSpotBound['b'] + l4 * maxSpotBound['s'] + l5 * minAmntBound['b'] + l6 * minAmntBound['s'] + l7 * maxAmntBound['b'] + l8 * maxAmntBound['s']
if simp:
    L = simplify(L)
L

In [ ]:
# Partial derivatives
dL_dDelta_s = L.diff(Delta_s)
dL_dDelta_b = L.diff(Delta_b)

# Set them to zero
stationary_conditions = [
    Eq(dL_dDelta_s, 0),
    Eq(dL_dDelta_b, 0),
]

In [ ]:
solutions_ = solve(stationary_conditions, (Delta_s, Delta_b, l1, l2, l3, l4, l5, l6, l7, l8, l9))

print(len(solutions_))
solutions = solutions_[0]
e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol, l7_sol, l8_sol, l9_sol = solutions
solutions

In [ ]:
for sol in solutions:
  print(sol.free_symbols)

In [ ]:
e_s_sol

In [ ]:
e_b_sol

In [ ]:
l1_sol

In [ ]:
l2_sol

In [ ]:
l3_sol

In [ ]:
l4_sol

In [ ]:
l5_sol

In [ ]:
l6_sol

In [ ]:
l7_sol

In [ ]:
l8_sol

In [ ]:
l9_sol